### Iris data
* 붓꽃 정보를 모아서 3가지 붓꽃 종류중 어디에 해당하는지 찾는 문제
* Specics : target(Vercisiconor,Virginica,Setosa)
* Sepal.width : 꽃받침의 너비
* Sepal.Length : 꽃받침의 길이
* Petal.Width : 꽃잎의 너비
* Petal.Length : 꽃잎의 길이

In [17]:
import sklearn
import numpy as np

In [18]:
from sklearn.datasets import load_iris
datasets = load_iris()

In [19]:
x_data = datasets["data"]
x_data[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [20]:
y_data = datasets["target"]
y_data

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [23]:
y_data = y_data.reshape([-1,1])
y_data[:3]

array([[0],
       [0],
       [0]])

In [24]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(y_data)  
y_data = enc.transform(y_data).toarray()
y_data[-1]

array([0., 0., 1.])

In [25]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
x_data_minmax = min_max_scaler.fit_transform(x_data)
x_data_minmax[:3]

array([[0.22222222, 0.625     , 0.06779661, 0.04166667],
       [0.16666667, 0.41666667, 0.06779661, 0.04166667],
       [0.11111111, 0.5       , 0.05084746, 0.04166667]])

In [26]:
x_0 =np.ones(x_data_minmax.shape[0])
x_data_minmax = np.column_stack((x_0, x_data_minmax))

x_data_minmax[:3]

array([[1.        , 0.22222222, 0.625     , 0.06779661, 0.04166667],
       [1.        , 0.16666667, 0.41666667, 0.06779661, 0.04166667],
       [1.        , 0.11111111, 0.5       , 0.05084746, 0.04166667]])

In [27]:
weights = np.random.uniform(size=(3,5))
weights

array([[0.06945579, 0.73308631, 0.23304687, 0.13765474, 0.44425877],
       [0.09250219, 0.21917646, 0.29277414, 0.30584722, 0.37036042],
       [0.94274873, 0.28411987, 0.74580473, 0.09055524, 0.83548474]])

### Softmax

In [28]:
def softmax(z):
    e = np.exp(z)
    p = e / np.sum(np.exp(z), axis=1).reshape([-1,1])
    return p

In [29]:
z = x_data_minmax.dot(weights.T)
z.shape

(150, 3)

In [30]:
softmax(z[:5])

array([[0.20075881, 0.19182851, 0.60741268],
       [0.21013932, 0.20405216, 0.58580851],
       [0.20053901, 0.20079627, 0.59866472],
       [0.20108292, 0.2048929 , 0.59402418],
       [0.19602537, 0.19047206, 0.61350257]])

### CrosEntropy

In [31]:
def cross_entropy_function(y, x, weights):
    z = x_data_minmax.dot(weights.T)
    result = - np.sum(
                np.sum(
                    (y * np.log(softmax(z))), axis=1).reshape((-1,1))
                )
    return result 

In [32]:
cross_entropy_function(y_data,x_data_minmax,weights)

187.88874854706208

#### Weights update

In [33]:
def minimize_grdient(y, x, initial_weights, iterations = 500000, alpha=0.001):
    cost_history= []
    theta_history = []
    m = y.shape[0]
    theta = np.copy(initial_weights)
    
    number_of_classes = theta.shape[0]
    number_of_weights = theta.shape[1]
    
    for _ in range(iterations):
        original_theta = np.copy(theta)
        for k in range(number_of_classes):        
            for j in range(number_of_weights):
                partial_x = x[:, j]
                partial_entropy = y - softmax(x.dot(original_theta.T))
                theta[k][j]  = original_theta[k][j] + (
                    alpha* partial_entropy[:,k].dot(partial_x.T) ) /150
        if (_ % 10000) == 0:
            print(cross_entropy_function(y,x,theta)/150)
            cost_history.append(cross_entropy_function(y,x,theta))
    return theta, cost_history

            

In [34]:
# weights = minimize_grdient(y_data, x_data_minmax,weights)
theta, cost_history = minimize_grdient(y_data, x_data_minmax,weights)

1.2524114140580078
0.776572911345834
0.6236449121398867
0.5439203906313789
0.4933502629793146
0.4572997781827651
0.42965916015831207
0.4074151250945141
0.3888933324283496
0.37307843606703767
0.35931279279983175
0.3471481857992583
0.33626655191249233
0.3264346850167054
0.3174769107216115
0.3092578468502611
0.3016711117020067
0.29463168979874593
0.28807062848431236
0.2819312662689876
0.27616649496211515
0.27073673597473247
0.26560842024007403
0.26075282982403736
0.2561452035795633
0.2517640384237516
0.24759053749769966
0.24360816996663145
0.23980231662859136
0.23615998216056747
0.23266955961021737
0.22932063621431795
0.22610383217826444
0.22301066594833724
0.2200334409326768
0.2171651497059859
0.21439939255777668
0.21173030787953906
0.20915251237980967
0.20666104950222208
0.20425134472574502
0.20191916666740045
0.19966059310008544
0.19747198115243886
0.19534994108219475
0.1932913131154777
0.19129314692682442
0.18935268340218486
0.18746733838267382
0.18563468813274647


#### 랜덤하게 인덱스 뽑기

In [35]:
rand_index= np.random.randint(0,150,30)
rand_index

array([126,  70, 115,  83,  99,  90,   7,  53, 145,  42,  46, 113, 141,
        21,  67,  27,  57,  88, 120,  92,  90, 126,  59, 123,  37,  64,
        61,  79,  10, 100])

In [36]:
y_pred = np.argmax(softmax(x_data_minmax[rand_index].dot(theta.T)),axis=1) 
y_pred

array([2, 2, 2, 1, 1, 1, 0, 1, 2, 0, 0, 2, 2, 0, 1, 0, 1, 1, 2, 1, 1, 2,
       1, 2, 0, 1, 1, 1, 0, 2], dtype=int64)

In [37]:
y_true = np.argmax(y_data[rand_index],axis=1)
y_true

array([2, 1, 2, 1, 1, 1, 0, 1, 2, 0, 0, 2, 2, 0, 1, 0, 1, 1, 2, 1, 1, 2,
       1, 2, 0, 1, 1, 1, 0, 2], dtype=int64)

In [38]:
y_pred == y_true

array([ True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [39]:
sum(y_pred == y_true) / len(rand_index)

0.9666666666666667